# Predicting Like Polynomial Terms

Remember in Algebra how you had to combine like terms to simplify problems? 

You'd see expressions like `60 + 2x^3 - 6x + x^3 + 17x` in which there are **5** total terms but only **4** are "like terms". `2x^3` and `x^3` are like, and `-6x` and `17x` are like, while `60` doesn't have any like siblings.

Can we teach a model to predict that there are `4` like terms in the above expression?

Let's give it a shot using [Mathy](https://mathy.ai) to generate math problems and [thinc](https://github.com/explosion/thinc) to build a regression model that outputs the number of like terms in each input problem.

In [ ]:
!pip install thinc mathy

### Encode Text Inputs

Mathy generates ascii-math problem texts that we need to encode into scalar values for the model to process. We'll one-hot encode the text by individual characters to keep things simple.

For math problems our vocabulary will include all the characters of the alphabet, numbers 0-9, and the special characters used for operators like `*`, `/`, `.`, etc.

Thinc has a `to_categorical` helper that we can use for converting values into one-hot arrays. Let's use that to write a function that accepts an input string and returns a one-hot representation.

In [2]:
from thinc.types import Array2d
from thinc.api import Ops, get_current_ops, to_categorical

vocab = " .+-/^*()[]-01234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"

def encode_input(text: str) -> Array2d:
    ops: Ops = get_current_ops()
    vocab_indices: List[int] = [vocab.index(s) for s in text]
    return to_categorical(ops.asarray(vocab_indices), n_classes=len(vocab))


#### Try It

Let's try it out on some fixed data to be sure it works. 

We should expect to see an output of one-hot arrays, one for each character in the input.

In [3]:
outputs = encode_input("4+2")
assert outputs[0].argmax() == vocab.index("4")
assert outputs[1].argmax() == vocab.index("+")
assert outputs[2].argmax() == vocab.index("2")
print(outputs)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]


### Generate Math Problems

We'll use Mathy to generate random polynomial problems with a variable number of like terms. The generated problems will act as training data for our model.

In [4]:
from typing import List, Optional, Set
import random
from mathy.problems import gen_simplify_multiple_terms


def generate_problems(number: int, exclude: Optional[Set[str]] = None) -> List[str]:
    if exclude is None:
        exclude = set()
    problems: List[str] = []
    while len(problems) < number:
        text, _ = gen_simplify_multiple_terms(
            random.randint(3, 8), noise_probability=1.0, noise_terms=10
        )
        assert text not in exclude, "duplicate problem generated!"
        problems.append(text)
    return problems


#### Try It

In [5]:
generate_problems(10)

['12f + -6624y + 11.8x + 5x + 6.7x + -3139x + 10q + 948s + -8685o^4 + 5.8u^2 + -2833f + 7j + 5.9f + 12n + -8291l + 11.8f + 10d^4 + -736t',
 '11p^4 + 1811d + 3z^4 + 9.0h + -648o + 0.8z^4 + 6p^4 + 4.7a + 7.0y^3 + -678m^4 + 11f + 3.2b + 3061p^4 + 2t + 9s + -28z^4',
 '10r + 2h + -6335b + 5597f + -5569p^3 + 8.2l^2 + 6.6c + 2844.4n + 4.2w + -4266q + 1431.4p^3 + 1.5k^3 + 11.4v',
 '5n + 3q + 6.7s^2 + 1k + 5.8g + 8192.0w + 7.5a + 1.1u + 6c + 3z + 6m^2 + 1487z + -3267m^2 + 4.2z + 2313.0m^2 + -5745z + 12f^4',
 '4147.9p + -7487m + -6963.9t^4 + 1.8q + -7945r^3 + -7471.8t^4 + 11.5l + 5.6l + 10l + 0.3z + 5.4y^2 + 7.8n + -8982t^4 + 2.4l + 2.7s + 1o + 6.6v',
 '6j^4 + 1.9s + -6310r^3 + 11.4a + -995d + 8969v^2 + 5.5s + 1m^2 + 1007u^4 + 9879g^4 + 4.9s + 8x + 3u^4 + 6754o + 2.5n^4 + 10.5u^4',
 '8.6f + 7.3d + 5n + 9d + 9.1z^3 + 9164y + 0.9d + 7663u^3 + -4640a^2 + -2512j + 3604n + 10b^3 + 5.9n + 10w + 5217d + -4344t + 10q + -1014n',
 '12a^2 + 6t^2 + 11w^2 + -7833x + 2b + 7.4h^3 + 4z^3 + 2q^4 + -7406o + 8.8y 

### Count Like Terms

Now that we can generate input problems, we'll need a function that can count the like terms in each one and return the value for use as a label.

To accomplish this we'll use a few helpers from mathy to enumerate the terms and compare them to see if they're like.

In [6]:
from typing import Optional, List, Dict
from mathy import MathExpression, ExpressionParser, get_terms, get_term_ex, TermEx
from mathy.problems import mathy_term_string

parser = ExpressionParser()

def count_like_terms(input_problem: str) -> int:
    expression: MathExpression = parser.parse(input_problem)
    term_nodes: List[MathExpression] = get_terms(expression)
    node_groups: Dict[str, List[int]] = {}
    for term_node in term_nodes:
        ex: Optional[TermEx] = get_term_ex(term_node)
        assert ex is not None, f"invalid expression {ex}"
        key = mathy_term_string(variable=ex.variable, exponent=ex.exponent)
        if key == "":
            key = "const"
        if key not in node_groups:
            node_groups[key] = [term_node]
        else:
            node_groups[key].append(term_node)
    like_terms = 0
    for k, v in node_groups.items():
        if len(v) <= 1:
            continue
        like_terms += len(v)
    return like_terms

#### Try It

In [7]:
assert count_like_terms("4x + 2y + q") == 0
assert count_like_terms("x + x + z") == 2
assert count_like_terms("4x + 2x + x + 7") == 3

### Generate Problem/Answer pairs

Now that we can generate problems, count the number of like terms in them, and encode their text into one-hot vectors, we have the pieces required to generate random problems and answers that we can train a neural network on.

Let's write a function that will return a tuple of: the problem text, its encoded example form, and the output label.

In [8]:
from typing import Tuple
from thinc.types import Array2d

def to_example(input_problem: str) -> Tuple[str, Array2d, List[int]]:
    one_hot = encode_input(input_problem)
    like_terms = count_like_terms(input_problem)
    return input_problem, one_hot, [like_terms]

#### Try It

In [10]:
text, X, Y = to_example("x+2x")
# text is preserved for debugging/visualization
assert text == "x+2x"
# X contains one-hot vectors
assert X[0].argmax() == vocab.index("x")
# Y is the number of like terms
assert Y[0] == 2
print(text, X, Y)

x+2x [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]] [2]


### Build a Model

Now that we can generate X/Y values, let's define our model and verify it can process a single input/output.

For this we'll use Thinc and define a model that takes in an `Array2d` of examples and outputs an `Array1d` of scalar values, one for each example input.

In [11]:
from typing import List
from thinc.model import Model
from thinc.types import Array2d, Array1d
from thinc.api import chain, list2ragged, reduce_sum, ReLu


def to_ints():
    def forward(model: Model[Array2d, Array1d], Y: Array2d, is_train: bool):
        Y = Y.astype("i")
        return Y, lambda dY: dY.astype("f")

    return Model("toint", forward)


def CountLikeTerms(n_hidden: int, dropout: float = 0.5) -> Model[List[Array2d], Array2d]:
    return chain(
        list2ragged(),
        reduce_sum(),
        ReLu(n_hidden, dropout=dropout),
        ReLu(n_hidden),
        ReLu(n_hidden),
        ReLu(n_hidden),
        ReLu(1),
        to_ints(),
    )

#### Try It

Let's pass an example through the model to make sure we have all the sizes right.

In [12]:
text, X, Y = to_example("14x + 2y - 3x + 7x")
m = CountLikeTerms(12)
m.initialize([X], Y)
assert m.predict([X]).shape == (1, 1)

### Generate Training Datasets

Now that we can generate examples and we have a model that can process them, let's generate random unique training and evaluation datasets.

For this we'll write another helper function that can generate (n) training examples and respects an exclude list to avoid letting examples from the training/test sets overlap.

In [13]:
from typing import Optional, Set, List
from thinc.types import Array2d

def generate_dataset(
    size: int, exclude: Optional[Set[str]] = None
) -> Tuple[List[str], List[Array2d], List[List[int]]]:
    texts: List[str] = generate_problems(size)
    examples: List[Array2d] = []
    labels: List[List[int]] = []
    for i, text in enumerate(texts):
        text, x, y = to_example(text)
        examples.append(x)
        labels.append(y)

    return texts, examples, labels

#### Try It

Generate a small dataset to be sure everything is working as expected

In [14]:
generate_dataset(3)

(['4453j + -7872q + -1522.0o + 0.9n + 4a + 6k^4 + 8l + 11t + 9l + 12t + 9097s^2 + 10u + 1g^2 + 5290f^4',
  '2078z^3 + 7a + 0.1l + -1658k^3 + 8.4a + -1487.6z^3 + 8.9o^2 + -6395a + 9225a + 4938.2r^2 + 1g + 5663z^3 + 5x + 8733w + 10m + 9n^4 + 10y',
  '3177p^3 + 7730.4f^3 + 9.3f^3 + -901y^4 + 8.5c^2 + 7b + 5f^3 + 1p^3 + 7p^3 + 1g + 3071f^3 + 7a + 3t + 1978n^3 + 5.4k^2 + 11o^2 + 7h^4'],
 [array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0.

### Evaluate Model Performance

We're almost ready to train our model, we just need to write a function that will check a given trained model against a given dataset and return a 0-1 score of how accurate it was.

We'll use this function to print the score as training progresses and print final test predictions at the end of training.

In [16]:
from typing import List
from thinc.model import Model
from thinc.types import Array2d, Array1d
from wasabi import msg


def evaluate_model(
    model: Model[Array2d, Array1d],
    *,
    print_problems: bool = False,
    texts: List[str],
    X: Array2d,
    Y: List[int],
):
    Yeval = model.predict(X)
    correct_count = 0
    print_n = 10
    if print_problems:
        msg.divider(f"eval samples max({print_n})")
    for text, y_answer, y_guess in zip(texts, Y, Yeval):
        y_guess = round(float(y_guess))
        correct = y_guess == int(y_answer[0])
        print_fn = msg.fail
        if correct:
            correct_count += 1
            print_fn = msg.good
        if print_problems and print_n > 0:
            print_n -= 1
            print_fn(f"Answer[{y_answer[0]}] Guess[{y_guess}] Text: {text}")
    return correct_count / len(X)


#### Try It

Let's try it out with an untrained model and expect to see a really sad score.

In [17]:
texts, X, Y = generate_dataset(128)
m = CountLikeTerms(12)
m.initialize(X, Y)
# Assume the model should do so poorly as to round down to 0
assert round(evaluate_model(m, texts=texts, X=X, Y=Y)) == 0

### Train the Model


In [24]:
from typing import Set
from thinc.api import Adam, fix_random_seed
from wasabi import msg
import numpy

# Set the random seed to make results more reproducible
fix_random_seed(1234)
batch_size = 12
num_iters = 12
train_size = 50000
test_size = 128
seen_texts: Set[str] = set()
with msg.loading(f"Generating train dataset with {train_size} examples..."):
    (train_texts, train_X, train_y) = generate_dataset(train_size, seen_texts)
msg.good(f"Train set created with {train_size} examples.")
with msg.loading(f"Generating eval dataset with {test_size} examples..."):
    (eval_texts, eval_X, eval_y) = generate_dataset(test_size, seen_texts)
msg.good(f"Eval set created with {test_size} examples.")
model = CountLikeTerms(512)
model.initialize(train_X[:2], model.ops.asarray(train_y[:2]))
optimizer = Adam(1e-3)
for n in range(num_iters):
    loss = 0.0
    batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
    for X, Y in batches:
        Y = model.ops.asarray(Y, dtype="float32")
        Yh, backprop = model.begin_update(X)
        err = Yh - Y
        backprop(err)
        loss += (err ** 2).sum()
        model.finish_update(optimizer)
    score = evaluate_model(model, texts=eval_texts, X=eval_X, Y=eval_y)
    print(f"{n}\t{score:.2f}\t{loss:.2f}")

Yeval = model.predict(eval_X)
score = evaluate_model(
    model, print_problems=True, texts=eval_texts, X=eval_X, Y=eval_y
)
print(f"Score: {score}")


nerating train dataset with 50000 examples...✔ Train set created with 50000 examples.
nerating eval dataset with 128 examples...✔ Eval set created with 128 examples.
0	0.48	127295.00
1	0.46	48669.00
2	0.50	32066.00
3	0.60	25133.00
4	0.72	21085.00
5	0.67	17909.00
6	0.74	15972.00
7	0.74	14372.00
8	0.69	13050.00
9	0.80	12357.00
10	0.80	11569.00
11	0.82	10946.00

============================ eval samples max(10) ============================
✘ Answer[[4]] Guess[3] Text: 6.9k + -4124l^4 + 4d + 10m^3 + 0.2x + 6q +
-7098p^2 + 7.3l^4 + 4o + 4r + 5975v^4 + 11s^2 + 0.4g^3 + 10.4g^3
✔ Answer[[4]] Guess[4] Text: 2510g + -8759m + 12t + 9y + 0.5c + 1880a +
10.9j^4 + 1q + 1143.9c + 4.0n^4 + 3.7y + 8356s + 5u^3 + 5.0x
✔ Answer[[2]] Guess[2] Text: 9h + 1.8n^3 + 11.0z + 7969w^4 + -4606j +
3604v^3 + 5573.4g + -8408u + 4s^4 + 2u + 6f + 10x^2 + 6p^2
✔ Answer[[7]] Guess[7] Text: 8z + 9m + 6.4y + -7238.6d^2 + 9.0z + 11f +
6u^4 + 2w^4 + 12l + 1022d^2 + 4t + 8335x + 7.6z + 8430n + -6979z + 9004c^2 +
-6079d^2
✔ 

### Advanced Exercise

Rewrite the model to encode the whole expression with a BiLSTM, and then generate pairs of terms, using the BiLSTM vectors. Over each pair of terms, predict whether the terms are alike or unlike.

In [ ]:
from dataclasses import dataclass
from thinc.types import Array2d, Ragged
from thinc.model import Model


@dataclass
class Comparisons:
    data: Array2d  # Batch of vectors for each pair
    indices: Array2d  # Int array of shape (N, 3), showing the (batch, term1, term2) positions

def pairify() -> Model[Ragged, Comparisons]:
    """Create pair-wise comparisons for items in a sequence. For each sequence of N
    items, there will be (N**2-N)/2 comparisons."""
    ...

def predict_over_pairs(model: Model[Array2d, Array2d]) -> Model[Comparisons, Comparisons]:
    """Apply a prediction model over a batch of comparisons. Outputs a Comparisons
    object where the data is the scores. The prediction model should predict over
    two classes, True and False."""
    ...
